In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import re

In [2]:
import spacy 
nlp = spacy.load("en_core_web_sm")

In [11]:
data = pd.read_csv('data.csv', sep = "\t", index_col = 0)
contestants = pd.read_csv('contestants.csv')

In [19]:
contestants

,year,to_country_id,to_country,performer,song,place_contest,sf_num,running_final,running_sf,place_final,...,points_sf,points_tele_final,points_jury_final,points_tele_sf,points_jury_sf,composers,lyricists,lyrics,youtube_url,is_last
325,1977,fr,France,Marie Myriam,L'oiseau Et L'enfant,1.0,NaN,18.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,Jean-Paul Cara,Joe Gracy,Comme un enfant aux yeux de lumière\nQui voit ...,https://youtube.com/watch?v=ny_YsJUVdBg,False
342,1977,se,Sweden,Forbes,Beatles,18.0,NaN,13.0,NaN,18.0,...,NaN,NaN,NaN,NaN,NaN,Claes Bure,Sven-Olof Bagge,"Beatles gav oss sin musik, 'Yesterday', vad de...",https://youtube.com/watch?v=OrFg4nyBeAE,True
343,1978,il,Israel,Izhar Cohen and the Alphabeta,A-Ba-Ni-Bi,1.0,NaN,18.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,Nurit Hirsh,Ehud Manor,A-ba-ni-bi o-bo-he-bev\nA-ba-ni-bi o-bo-he-bev...,https://youtube.com/watch?v=QvZS6tnZKMU,False
362,1978,no,Norway,Jahn Teigen,Mil Etter Mil,20.0,NaN,2.0,NaN,20.0,...,NaN,NaN,NaN,NaN,NaN,Kai Eide,NaN,"Hvil deg nå, du er sliten\nHvil din kropp før ...",https://youtube.com/watch?v=dVj8tlXQ1Vg,True
363,1979,il,Israel,Milk and Honey,Hallelujah,1.0,NaN,10.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,Kobi Oshrat,Shimrit Orr,"Hallelujah la'olam\nHallelujah, yashiru kulam\...",https://youtube.com/watch?v=C33kO3fvjkI,False
380,1979,be,Belgium,Micha Marah,Hey Nana,18.0,NaN,12.0,NaN,18.0,...,NaN,NaN,NaN,NaN,NaN,Charles Dumolin,Guy Beyers,"Hé, kom je mee?\nBlijf je hier bij mij vannach...",https://youtube.com/watch?v=kHKjdkduPNI,True
381,1979,at,Austria,Christina Simon,Heute In Jerusalem,18.0,NaN,18.0,NaN,18.0,...,NaN,NaN,NaN,NaN,NaN,Peter Wolf,André Heller,"Erhebt euch, tut ab den Schlaf\nIn dieser Zeit...",https://youtube.com/watch?v=X3w26dtqpZY,True
382,1980,ie,Ireland,Johnny Logan,What's Another Year,1.0,NaN,17.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,Shay Healy,NaN,I've been waitin' such a long time\nLookin' ou...,https://youtube.com/watch?v=BPOwTXWe-Yw,False
400,1980,fi,Finland,Vesa-Matti Loiri,Huilumies,19.0,NaN,10.0,NaN,19.0,...,NaN,NaN,NaN,NaN,NaN,Aarno Raninen,Veikko 'Vexi' Salmi,On tässä huilu ja huilumies\nEn täytä mittaa n...,https://youtube.com/watch?v=MaBhx84GA8g,True
401,1981,gb,United Kingdom,Bucks Fizz,Making Your Mind Up,1.0,NaN,14.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,John Danter,Andy Hill,You gotta speed it up\nAnd then you gotta slow...,https://youtube.com/watch?v=62SrA5hCmkE,False


In [12]:
last_places = {}
for year in list(set(list(contestants["year"]))):
    last_places[year] = max(list(contestants[contestants["year"] == year]["place_contest"]))
#print(last_places)

is_last = []
for index, row in contestants.iterrows():
    if row["place_contest"] == last_places[row["year"]]:
        is_last.append(True)
    else:
        is_last.append(False)
        
#print(is_last)

contestants["is_last"] = is_last

In [13]:
contestants = contestants[(contestants['place_contest'] == 1) | contestants['is_last']]

In [18]:
contestants = contestants[(contestants['year'] > 1976) & (contestants['year'] < 1998)]
# during this time, songs had to be in the country's official language

In [29]:
data[data['song'] == 'Nocturne']['spotify_url'].values[0]

'https://open.spotify.com/embed/track/6bk11TH82XrT5QUC1gDw9A?utm_source=generator'

In [39]:
spotify_links = {}

for index, row in contestants.iterrows():
    if row['song'] in list(data['song']):
        spotify_links[index] = data[data['song'] == row['song']]['spotify_url'].values[0]
    else:
        link = input(row['song'] + " " + row['performer'])
        spotify_links[index] = link
        
print(spotify_links)

L'oiseau Et L'enfant Marie Myriamhttps://open.spotify.com/embed/track/7Hy0CZtFcREucLDXB5jp6m?utm_source=generator
Beatles Forbeshttps://open.spotify.com/embed/track/1LiPcvWAEMC4AMUnHDbppP?utm_source=generator
A-Ba-Ni-Bi Izhar Cohen and the Alphabeta?
Mil Etter Mil Jahn Teigenhttps://open.spotify.com/embed/track/2tbNnbDM8vcGxlPqzyQHNj?utm_source=generator
Hallelujah Milk and Honey?
Hey Nana Micha Marah?
Heute In Jerusalem Christina Simon-
What's Another Year Johnny Loganhttps://open.spotify.com/embed/track/3xGXaTsKtS55UZiWgcXlu8?utm_source=generator
Huilumies Vesa-Matti Loirihttps://open.spotify.com/embed/track/4eyxjL6wFEfF0xudWnFr32?utm_source=generator
Making Your Mind Up Bucks Fizzhttps://open.spotify.com/embed/track/0Azn2vpP3Sbs8qAG8e1KUc?utm_source=generator
Aldri i Livet Finn Kalvikhttps://open.spotify.com/embed/track/5r6BaTfb8fODI4he90GFhr?utm_source=generator
Ein Bißchen Frieden Nicolehttps://open.spotify.com/embed/track/5ZSfGGi8OF1vSSVNAWdQT1?utm_source=generator
Nuku Pommiin K

In [42]:
contestants["spotify_links"] = pd.Series(spotify_links)

/var/folders/ky/d01ywblx1l5fk6g19868_t7m0000gn/T/ipykernel_17720/1196979150.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contestants["spotify_links"] = pd.Series(spotify_links)


In [111]:
def get_prompt(song, p):
    
    print(f'You are an AI that is able to predict whether a song will be a winner or a loser at the eurovision song context, based on the title, singer, country of origin, and the lyrics.') 
          
    print(f'You predict that the following song called "{song["song"]}" by {song["performer"]} from {song["to_country"]} will be a {p}.')
    print("These are the lyrics of that song:")
    print()
    print('"' + song["lyrics"].replace('\\n', '\n') + '"')
    
    lines = sorted(list(set([l.strip() for l in song["lyrics"].replace('\\n', '\n').split("\n") if not l.strip() == ""])))
    x = random.choice(lines)
    
    print()
    print(f'The part of the lyrics that had the biggest influence on your prediction is the line "{x}". Write a natural language explanation of your prediction.')
    
    gpt_result = input('')
    return gpt_result

In [125]:
ai_predictions = {}

for index, row in list(contestants.to_dict('index').items()):
    
    if index in ai_predictions_first_batch:
        ai_predictions[index] = ai_predictions_first_batch[index]
    else:
    
        true_prediction = not row["is_last"]
        p = ['loser', 'winner']
        ai_true = ""
        ai_false = ""

        # correct explanation 
        if row['song'] in list(data['song']) and not str(data[data['song'] == row['song']]['explanation_correct'].values[0]) == "nan":
            ai_true = data[data['song'] == row['song']]['explanation_correct'].values[0]
        else:
            ai_true = get_prompt(row, p[true_prediction])

        print("\n-----------------\n")

        # incorrect explanation
        if row['song'] in list(data['song'])and not str(data[data['song'] == row['song']]['explanation_incorrect'].values[0]) == "nan":
            ai_false = data[data['song'] == row['song']]['explanation_incorrect'].values[0]
        else:
            ai_false = get_prompt(row, p[not true_prediction])

        print("\n\n\n-----------------------------------------\n\n\n")

        ai_predictions[index] = {'explanation_correct':ai_true, 'explanation_incorrect':ai_false}
        
                             
                             
            
        




You are an AI that is able to predict whether a song will be a winner or a loser at the eurovision song context, based on the title, singer, country of origin, and the lyrics.
You predict that the following song called "Mil Etter Mil" by Jahn Teigen from Norway will be a loser.
These are the lyrics of that song:

"Hvil deg nå, du er sliten
Hvil din kropp før din kraft ebber ut
Vann pipler inn i dine sko
Du vandrer i regn

Mil etter mil etter mil
Etter mil etter mil etter mil

Søk mot meg hvis du lengter
Rett ditt sinn mot et annet sted
Komm hit hvor jeg er, hvor solen er nær
Du må ikke gå

Mil etter mil etter mil
Etter mil etter mil etter mil

Og det er langt å gå
Det er lengselen som driver deg
Mot et lys som du kan se
Men ikke jeg

Tenk deg om nå
Hvor farer du hen
Du har vandret altfor lenge

Mil etter mil etter mil
Etter mil etter mil etter mil

Tenk deg om nå
Hvor farer du hen
Du har vandret altfor lenge

Mil etter mil etter mil
Etter mil etter mil etter mil
Etter mil etter mil ett

Based on the lyrics of the song "Hey Nana" by Micha Marah from Belgium, the line "Zo zie je maar" had the biggest influence on my prediction that this song would be a loser. This line roughly translates to "See, that's how it goes" or "That's how it is". This line, along with the overall theme of the song which is about a woman hoping to settle down with a man and live a simple life, suggests that the song lacks originality and has a somewhat generic theme. Additionally, the simplistic lyrics and repetitive nature of the song may not be enough to captivate the audience and stand out among the other entries in the Eurovision song context.

-----------------

You are an AI that is able to predict whether a song will be a winner or a loser at the eurovision song context, based on the title, singer, country of origin, and the lyrics.
You predict that the following song called "Hey Nana" by Micha Marah from Belgium will be a winner.
These are the lyrics of that song:

"Hé, kom je mee?
Blijf


-----------------

You are an AI that is able to predict whether a song will be a winner or a loser at the eurovision song context, based on the title, singer, country of origin, and the lyrics.
You predict that the following song called "What's Another Year" by Johnny Logan from Ireland will be a loser.
These are the lyrics of that song:

"I've been waitin' such a long time
Lookin' out for you, but you're not here
What's another year?

I've been wakin' such a long time
Reachin' out for you, but you aren't near
What's another year?

What's another year
For someone who's lost everything that he owns?
What's another year
For someone who's gettin' used to bein' alone?

I've been prayin' such a long time
It's the only way to hide the fear
What's another year?

What's another year
For someone who's lost everything that he owns?
What's another year
For someone who's gettin' used to bein' alone?

I've been cryin' such a long time
With such a lot of pain in every tear
What's another year
(For

Based on the line "And now you really gotta speed it up" in the lyrics, I predict that the song "Making Your Mind Up" by Bucks Fizz from the United Kingdom will be a winner at the Eurovision song contest. The upbeat tempo and the catchy chorus of the song, along with the message of taking risks and making decisions, are likely to appeal to the audience and the judges. The lyrics suggest that the song has a high energy level and encourages the listener to get up and dance. Additionally, the repetition of the line "For making your mind up" throughout the song reinforces the message of making a decision and taking action, which could resonate with the Eurovision audience. Overall, the combination of the upbeat tempo and positive message make "Making Your Mind Up" a strong contender for a winning song.

-----------------

You are an AI that is able to predict whether a song will be a winner or a loser at the eurovision song context, based on the title, singer, country of origin, and the ly

Based on the lyrics and other features, I predict that "Ein Bißchen Frieden" by Nicole from Germany will be a winner at the Eurovision Song Contest. One of the reasons for this prediction is the line "Ich weiss, meine Lieder, die ändern nicht viel" which translates to "I know my songs don't change much". This line is powerful because it captures a universal sentiment that many people can relate to, which is the feeling of helplessness and smallness in the face of big problems in the world. The song offers a message of hope and peace, and the sentiment is conveyed through the lyrics, the melody, and the singer's performance. Additionally, the song is catchy and uplifting, and it has a strong chorus that is easy to remember and sing along to. All these factors make me predict that this song will be a winner at the Eurovision Song Contest.

-----------------

You are an AI that is able to predict whether a song will be a winner or a loser at the eurovision song context, based on the title

...

Based on the provided lyrics, my prediction is that "Brandenburger Tor" by Ketil Stokkan from Norway will be a loser at the Eurovision song contest. The line "Hånd i hånd som om det va' i går" doesn't convey a strong emotional message that is likely to resonate with the audience. Additionally, the lyrics are quite repetitive and lack a clear and memorable chorus, which are important elements of successful Eurovision songs. Furthermore, the song doesn't seem to have a unique or standout quality that would make it stand out from the other entries in the contest. Overall, these factors lead me to predict that "Brandenburger Tor" will not perform well in the competition.

-----------------




-----------------------------------------




-----------------

You are an AI that is able to predict whether a song will be a winner or a loser at the eurovision song context, based on the title, singer, country of origin, and the lyrics.
You predict that the following song called "Fri?" by Beat fr

Based on the given lyrics, my prediction is that the song "Why Me" by Linda Martin from Ireland will be a loser at the Eurovision song contest. The lyrics of the song do not contain any strong or unique phrases that suggest it will be a winner. Additionally, the line "Sometimes I watch you passing by my window" does not carry a significant emotional weight that would captivate the audience or the judges. Overall, the lyrics of the song do not appear to have a strong impact, and therefore, I predict it will be a loser at the competition.



-----------------------------------------



You are an AI that is able to predict whether a song will be a winner or a loser at the eurovision song context, based on the title, singer, country of origin, and the lyrics.
You predict that the following song called "Yamma Yamma" by Pave from Finland will be a loser.
These are the lyrics of that song:

"(Yamma yamma yamma yamma yamma yamma yamma ya...)
(Yamma yamma yamma yamma yamma yamma yamma ya...)



Based on the title, lyrics, and other factors, my prediction is that "Rock 'n' Roll Kids" by Paul Harrington & Charlie McGettigan from Ireland will be a winner at the Eurovision Song Contest. The line "And we never knew what life held in store" suggests a sense of nostalgia for a simpler time, when rock 'n' roll was all they did and life was full of possibilities. This nostalgic tone may resonate with audiences, especially those who grew up during the rock 'n' roll era, and create a sentimental connection that could lead to a winning performance. Additionally, the upbeat rhythm and catchy melody of the song may also contribute to its potential success.

-----------------




-----------------------------------------




-----------------

You are an AI that is able to predict whether a song will be a winner or a loser at the eurovision song context, based on the title, singer, country of origin, and the lyrics.
You predict that the following song called "Lopšinė mylimai" by Ovidijus Vy




-----------------------------------------




-----------------

You are an AI that is able to predict whether a song will be a winner or a loser at the eurovision song context, based on the title, singer, country of origin, and the lyrics.
You predict that the following song called "Niin Kaunis On Taivas" by Jasmine from Finland will be a winner.
These are the lyrics of that song:

"Niin kaunis on taivas, kaunis on maa
Me tähtien loisteessa yö kuljetaan
Rannalle kun käsikkäin saavutaan
Luonas olla tahdon vaan

Joka sekunnin, jonka kanssas olla saan
Joka minuutin, tunnen yhä varmemmin
Rakkaus ikuinen, kohtalomme yhteinen
Se uni suloinen, kuin on

Niin kaunis on taivas, kaunis on maa
Me tähtien loisteessa yö kuljetaan
Rannalle kun käsikkäin saavutaan
Luonas olla tahdon vaan

Joka sekunnin, jonka luonas olla saan
Joka minuutin, tunnen yhä paremmin
Voima tunteiden, meille sykkii rakkauden
Se uni suloinen kuin on, on

Niin kaunis on taivas, kaunis on maa
Me tähtien loisteessa yö kuljeta

!!!

-----------------




-----------------------------------------





In [128]:
ai_correct = {i:e['explanation_correct'] for i, e in ai_predictions.items()}
ai_incorrect = {i:e['explanation_incorrect'] for i, e in ai_predictions.items()}

In [129]:
contestants['explanation_correct'] = pd.Series(ai_correct)

/var/folders/ky/d01ywblx1l5fk6g19868_t7m0000gn/T/ipykernel_17720/2223069176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contestants['explanation_correct'] = pd.Series(ai_correct)


In [130]:
contestants['explanation_incorrect'] = pd.Series(ai_incorrect)

/var/folders/ky/d01ywblx1l5fk6g19868_t7m0000gn/T/ipykernel_17720/1898596379.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contestants['explanation_incorrect'] = pd.Series(ai_incorrect)


In [131]:
contestants

,year,to_country_id,to_country,performer,song,place_contest,sf_num,running_final,running_sf,place_final,...,points_tele_sf,points_jury_sf,composers,lyricists,lyrics,youtube_url,is_last,spotify_links,explanation_correct,explanation_incorrect
325,1977,fr,France,Marie Myriam,L'oiseau Et L'enfant,1.0,NaN,18.0,NaN,1.0,...,NaN,NaN,Jean-Paul Cara,Joe Gracy,Comme un enfant aux yeux de lumière\nQui voit ...,https://youtube.com/watch?v=ny_YsJUVdBg,False,https://open.spotify.com/embed/track/7Hy0CZtFc...,Based on the analysis of the lyrics of the son...,As an AI model trained to predict Eurovision S...
342,1977,se,Sweden,Forbes,Beatles,18.0,NaN,13.0,NaN,18.0,...,NaN,NaN,Claes Bure,Sven-Olof Bagge,"Beatles gav oss sin musik, 'Yesterday', vad de...",https://youtube.com/watch?v=OrFg4nyBeAE,True,https://open.spotify.com/embed/track/1LiPcvWAE...,Based on the analysis of the lyrics of the son...,As an AI model trained to predict Eurovision S...
343,1978,il,Israel,Izhar Cohen and the Alphabeta,A-Ba-Ni-Bi,1.0,NaN,18.0,NaN,1.0,...,NaN,NaN,Nurit Hirsh,Ehud Manor,A-ba-ni-bi o-bo-he-bev\nA-ba-ni-bi o-bo-he-bev...,https://youtube.com/watch?v=QvZS6tnZKMU,False,?,"As an AI, I predict that ""A-Ba-Ni-Bi"" by Izhar...","Based on the given lyrics, the line ""Hametukot..."
362,1978,no,Norway,Jahn Teigen,Mil Etter Mil,20.0,NaN,2.0,NaN,20.0,...,NaN,NaN,Kai Eide,NaN,"Hvil deg nå, du er sliten\nHvil din kropp før ...",https://youtube.com/watch?v=dVj8tlXQ1Vg,True,https://open.spotify.com/embed/track/2tbNnbDM8...,"Based on the line ""Tenk deg om nå"" in the lyri...","Based on the line ""Du har vandret altfor lenge..."
363,1979,il,Israel,Milk and Honey,Hallelujah,1.0,NaN,10.0,NaN,1.0,...,NaN,NaN,Kobi Oshrat,Shimrit Orr,"Hallelujah la'olam\nHallelujah, yashiru kulam\...",https://youtube.com/watch?v=C33kO3fvjkI,False,?,"Based on the lyrics of the song ""Hallelujah"" b...","Based on the provided lyrics of ""Hallelujah"" b..."
380,1979,be,Belgium,Micha Marah,Hey Nana,18.0,NaN,12.0,NaN,18.0,...,NaN,NaN,Charles Dumolin,Guy Beyers,"Hé, kom je mee?\nBlijf je hier bij mij vannach...",https://youtube.com/watch?v=kHKjdkduPNI,True,?,"Based on the lyrics of the song ""Hey Nana"" by ...","Based on the title and lyrics of ""Hey Nana"" by..."
381,1979,at,Austria,Christina Simon,Heute In Jerusalem,18.0,NaN,18.0,NaN,18.0,...,NaN,NaN,Peter Wolf,André Heller,"Erhebt euch, tut ab den Schlaf\nIn dieser Zeit...",https://youtube.com/watch?v=X3w26dtqpZY,True,-,"Based on the lyrics of ""Heute In Jerusalem"" by...","Based on the title ""Heute In Jerusalem"" and th..."
382,1980,ie,Ireland,Johnny Logan,What's Another Year,1.0,NaN,17.0,NaN,1.0,...,NaN,NaN,Shay Healy,NaN,I've been waitin' such a long time\nLookin' ou...,https://youtube.com/watch?v=BPOwTXWe-Yw,False,https://open.spotify.com/embed/track/3xGXaTsKt...,"The line ""With such a lot of pain in every tea...","Based on the lyrics of the song ""What's Anothe..."
400,1980,fi,Finland,Vesa-Matti Loiri,Huilumies,19.0,NaN,10.0,NaN,19.0,...,NaN,NaN,Aarno Raninen,Veikko 'Vexi' Salmi,On tässä huilu ja huilumies\nEn täytä mittaa n...,https://youtube.com/watch?v=MaBhx84GA8g,True,https://open.spotify.com/embed/track/4eyxjL6wF...,"Based on the lyrics of the song ""Huilumies"" by...","Based on the line ""Kaiken sen minkä aistin hui..."
401,1981,gb,United Kingdom,Bucks Fizz,Making Your Mind Up,1.0,NaN,14.0,NaN,1.0,...,NaN,NaN,John Danter,Andy Hill,You gotta speed it up\nAnd then you gotta slow...,https://youtube.com/watch?v=62SrA5hCmkE,False,https://open.spotify.com/embed/track/0Azn2vpP3...,"Based on the line ""And now you really gotta sp...","Based on the lyrics of ""Making Your Mind Up"" b..."


In [132]:
contestants.to_csv("data.csv", sep = "\t")